In [9]:
from datetime import datetime

from google.cloud import bigquery

In [10]:
client = bigquery.Client()

In [11]:
project_id = 'agentml'
dataset_id = 'training_data'
table_id = 'training_text'

table_ref = f"{project_id}.{dataset_id}.{table_id}"

In [12]:
table_ref

'agentml.training_data.training_text'

In [13]:
table = client.get_table(table_ref)

In [14]:
datetime.now()


datetime.datetime(2023, 4, 19, 14, 11, 21, 499529)

In [10]:
datetime.now().isoformat()

'2023-04-19T13:52:28.164730'

In [15]:
new_row = {
    "text": "hello 2",
    "created_at": datetime.now().isoformat(),
}


In [16]:
errors = client.insert_rows_json(table, [new_row])

In [17]:
errors

[{'index': 0,
  'errors': [{'reason': 'invalid',
    'location': '',
    'debugInfo': '',
    'message': 'Missing required fields: Msg_0_CLOUD_QUERY_TABLE.is_deleted, Msg_0_CLOUD_QUERY_TABLE.uuid.'}]}]

In [1]:
import uuid
uuid.uuid4()


UUID('de27dc2c-109a-47d3-affe-3aea5b4fc13c')

In [8]:
len(str(uuid.uuid4()))

36

In [4]:
uuid.uuid4().

95960628731042371618640198562667613156

In [18]:
uuids = [str(uuid.uuid4()) for _ in range(10)]

In [19]:
uuids

['9fc8b0fb-7dab-44d5-8764-76e603762840',
 'cdfc4c5d-7ddf-49bf-9c8f-bcc95309dc2f',
 '4a0ecbf7-44c9-4912-967b-84cc667e12b0',
 'b24ad3e7-bba6-4c9d-be6f-00daaf9f52e3',
 '08e7397f-5bb8-4383-9880-6bcef57f30f5',
 'da05f120-0a88-410b-a785-d4a7f79eb028',
 '68ba3afe-891e-4471-9e2a-bee6119382c9',
 'fe9d30d5-c2dc-4ce4-9a57-adee2ebd7362',
 'cc092359-7dc5-44a2-827d-0f8942253bc2',
 '359f4501-48f0-4a5b-b7ba-5ae4cdaef627']

In [20]:
', '.join(f"'{id_}'" for id_ in uuids)

"'9fc8b0fb-7dab-44d5-8764-76e603762840', 'cdfc4c5d-7ddf-49bf-9c8f-bcc95309dc2f', '4a0ecbf7-44c9-4912-967b-84cc667e12b0', 'b24ad3e7-bba6-4c9d-be6f-00daaf9f52e3', '08e7397f-5bb8-4383-9880-6bcef57f30f5', 'da05f120-0a88-410b-a785-d4a7f79eb028', '68ba3afe-891e-4471-9e2a-bee6119382c9', 'fe9d30d5-c2dc-4ce4-9a57-adee2ebd7362', 'cc092359-7dc5-44a2-827d-0f8942253bc2', '359f4501-48f0-4a5b-b7ba-5ae4cdaef627'"

In [22]:
from prodgpt.data.data_warehouse import DataWarehouse
from prodgpt.utils import load_yaml

In [23]:
config = load_yaml("./configs/default.yaml")

In [24]:
data_warehouse = DataWarehouse(config)

In [25]:
uuids = ["b43b22b8-9d4f-4db9-a639-d754084c2a33"]

In [26]:
data_warehouse.extract_from_uuid(uuids)

AttributeError: 'Table' object has no attribute 'query'

In [27]:
data_warehouse.client.query(data_warehouse.table)

TypeError: Object of type Table is not JSON serializable

In [28]:
from sqlalchemy import create_engine, select, and_
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, DateTime, Boolean
from typing import Optional, List
from datetime import datetime

Base = declarative_base()


/var/folders/pf/g3nr86yn4j71vzzmv6knwdhr0000gp/T/ipykernel_11766/1665340599.py:8: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [29]:
class MyTable(Base):
    __tablename__ = 'training_text'
    id = Column(String, primary_key=True)
    timestamp_column = Column(DateTime)
    is_deleted = Column(Boolean)


In [30]:
class MyTable(Base):
    __tablename__ = 'my_table'
    id = Column(String, primary_key=True)
    timestamp_column = Column(DateTime)
    is_deleted = Column(Boolean)


class BigQueryExtractor:
    def __init__(self, project_id: str, dataset_id: str):
        self.engine = create_engine(f"bigquery://{project_id}/{dataset_id}")
        self.session = sessionmaker(bind=self.engine)()

    def extract(self, uuids: Optional[List[str]] = None, date: Optional[datetime] = None, is_deleted: Optional[bool] = None):
        query = select([MyTable])

        conditions = []

        if uuids:
            conditions.append(MyTable.id.in_(uuids))

        if date:
            date_str = date.strftime('%Y-%m-%d')
            conditions.append(MyTable.timestamp_column.cast(
                String).like(f"{date_str}%"))

        if is_deleted is not None:
            conditions.append(MyTable.is_deleted == is_deleted)

        if conditions:
            query = query.where(and_(*conditions))

        results = self.session.execute(query)
        data = [row for row in results]
        return data


/var/folders/pf/g3nr86yn4j71vzzmv6knwdhr0000gp/T/ipykernel_11766/2801392009.py:1: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.MyTable, and will be replaced in the string-lookup table.
  class MyTable(Base):


In [1]:
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
engine = create_engine('bigquery://agentml')

In [2]:
table = Table('training_data.training_text', MetaData(bind=engine), autoload=True)

/var/folders/pf/g3nr86yn4j71vzzmv6knwdhr0000gp/T/ipykernel_15191/3575246210.py:1: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  table = Table('training_data.training_text', MetaData(bind=engine), autoload=True)


In [3]:
print(select([func.count('*')], from_obj=table).scalar())

18
